In [2]:
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [8]:
file1 = '/home/jovyan/Data/pd3/pd_id_2019_agg3_3h.csv'
df1 = pd.read_csv(file1)
# df1.columns = ['geohash_3', 'no_of_points', 'no_of_unique_users', 'local_date']
df1['local_date'] = df1['local_date'].astype(int)
combined_df=df1
df1

,geohash_3,no_of_points,no_of_unique_users,local_time,local_date
0,qpz,2329,226,20190101 00:00:00 - 03:00:00,20190101
1,qqe,30,11,20190101 00:00:00 - 03:00:00,20190101
2,qqg,158085,6785,20190101 00:00:00 - 03:00:00,20190101
3,qqs,1973,271,20190101 00:00:00 - 03:00:00,20190101
4,qqt,4005,433,20190101 00:00:00 - 03:00:00,20190101
...,...,...,...,...,...
365643,wb0,2481,187,20191231 21:00:00 - 24:00:00,20191231
365644,wb2,3770,306,20191231 21:00:00 - 24:00:00,20191231
365645,wb3,239,20,20191231 21:00:00 - 24:00:00,20191231
365646,wb4,857,67,20191231 21:00:00 - 24:00:00,20191231


In [20]:
# import os
# import pandas as pd

# # Specify the folder containing the files
# folder_path = '/home/jovyan/Data/Fixing'

# # Get a list of all files in the folder
# file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# # Initialize an empty DataFrame
# combined_df = pd.DataFrame()

# # Define the new column names
# new_columns = ['geohash_3', 'no_of_points', 'no_of_unique_users', 'local_date']

# # Loop through all files and append their content to the combined DataFrame
# for file in file_list:
#     file_path = os.path.join(folder_path, file)
#     df = pd.read_csv(file_path)
#     df.columns = new_columns  # Rename columns
#     combined_df = pd.concat([combined_df, df], ignore_index=True)

# combined_df

,geohash_3,no_of_points,no_of_unique_users,local_date
0,tgt,162348,2925,20190522
1,tgv,62093,985,20190522
2,tuj,189563,3555,20190522
3,tum,60512,1328,20190522
4,tut,103758,1879,20190522
...,...,...,...,...
1970,whe,10293,246,20190527
1971,whf,18868,444,20190527
1972,whg,60898,1412,20190527
1973,whu,8681,235,20190527


# Check duplicates

In [10]:
combined_df['geohash_3'] = combined_df['geohash_3'].astype(str)
combined_df['no_of_points'] = combined_df['no_of_points'].astype(int)
combined_df['no_of_unique_users'] = combined_df['no_of_unique_users'].astype(int)
combined_df['local_time'] = combined_df['local_time'].astype(str)
combined_df['local_date'] = combined_df['local_date'].astype(int)

combined_df = combined_df.drop_duplicates()
combined_df

,geohash_3,no_of_points,no_of_unique_users,local_time,local_date
0,qpz,2329,226,20190101 00:00:00 - 03:00:00,20190101
1,qqe,30,11,20190101 00:00:00 - 03:00:00,20190101
2,qqg,158085,6785,20190101 00:00:00 - 03:00:00,20190101
3,qqs,1973,271,20190101 00:00:00 - 03:00:00,20190101
4,qqt,4005,433,20190101 00:00:00 - 03:00:00,20190101
...,...,...,...,...,...
365643,wb0,2481,187,20191231 21:00:00 - 24:00:00,20191231
365644,wb2,3770,306,20191231 21:00:00 - 24:00:00,20191231
365645,wb3,239,20,20191231 21:00:00 - 24:00:00,20191231
365646,wb4,857,67,20191231 21:00:00 - 24:00:00,20191231


# Check missing dates

In [11]:
unique_values_count = combined_df["local_date"].unique()
all_dates = pd.date_range(start='2019-01-01', end='2019-12-31').strftime('%Y%m%d').astype(int)
all_dates = np.array(all_dates)
missing_dates = np.setdiff1d(all_dates, unique_values_count)

missing_dates

array([], dtype=int64)

# Check for hours per day (for 3h)

In [12]:
# Convert local_date to datetime objects
combined_df['local_date'] = pd.to_datetime(combined_df['local_date'], format='%Y%m%d')

# Function to parse the local_time column and extract the start hour
def parse_start_hour(time_str):
    try:
        start_time_str, _ = time_str.split(' - ')
        start_time = datetime.strptime(start_time_str, '%Y%m%d %H:%M:%S')
        return start_time.hour
    except ValueError:
        return None

combined_df['start_hour'] = combined_df['local_time'].map(parse_start_hour)

# Required hours to check
required_hours = set([0, 3, 6, 9, 12, 15, 18, 21])

# Group by local_date and check missing hours
missing_hours = {}

for date, group in combined_df.groupby('local_date'):
    present_hours = set(group['start_hour'])
    missing = required_hours - present_hours
    if missing:
        missing_hours[date] = sorted(list(missing))

# Convert missing hours dictionary to DataFrame for better readability
missing_hours_df = pd.DataFrame(list(missing_hours.items()), columns=['Date', 'Missing Hours'])

# Display the missing hours
print("Missing hours for each date:")
print(missing_hours_df)


/tmp/ipykernel_642/1032260670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['local_date'] = pd.to_datetime(combined_df['local_date'], format='%Y%m%d')


Missing hours for each date:
         Date Missing Hours
0  2019-01-18          [18]
1  2019-06-26          [18]
2  2019-07-10          [12]
3  2019-07-13          [18]
4  2019-07-26          [15]
5  2019-08-10           [9]
6  2019-08-27          [15]
7  2019-08-30           [0]
8  2019-09-20           [6]
9  2019-09-27          [21]
10 2019-09-28           [0]
11 2019-09-29          [18]
12 2019-10-09           [9]
13 2019-10-12           [9]
14 2019-10-13          [15]
15 2019-10-14          [18]
16 2019-11-01          [12]
17 2019-11-22          [18]
18 2019-12-20       [9, 12]


/tmp/ipykernel_642/1032260670.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['start_hour'] = combined_df['local_time'].map(parse_start_hour)


In [23]:
# Save the combined DataFrame to a new CSV file
combined_df.to_csv(folder_path + '/combined_file.csv', index=False)